In [ ]:
#default_exp datadownload

In [ ]:
#export
from kesscore.imports import *
from functools import wraps
import torchvision.datasets.utils as tvdu

import tarfile,zipfile,warnings,subprocess

In [ ]:
#export
def try_extract(fname, dest=None):
    "Extract `fname` to `dest` using `tarfile` or `zipfile`."
    if dest is None: dest = Path(fname).parent
    try:    tarfile.open(fname, 'r:gz').extractall(dest); return
    except: pass
    try:    zipfile.ZipFile(fname     ).extractall(dest); return
    except: pass
    raise Exception(f'Unrecognized archive: {fname}')

In [ ]:
# export
def _untar_data(url, name, dest, d_func, ext_func=try_extract):
    'Download to `dest/name.tar.gz` and extract to folder `dest/name`'
    dest = Path(dest)
    if (dest/name).exists(): warnings.warn('Folder already exist, so will not download again: ' + str(dest/name))
    else:
        fname = dest/f'{name}.tar.gz'
        if fname.exists(): warnings.warn(f'File already exist, will use it as is. ({fname})\n' +
                                         'If you wish, you might delete the file and run the program again instead.')
        else: d_func(url, fname)
        ext_func(fname, dest=dest/name)
        fname.unlink()
    return dest/name

In [ ]:
#export
def _drive_dl(fid, fname):
    tvdu.download_file_from_google_drive(file_id=fid, root=fname.parent, filename=fname.name)

In [ ]:
#export
@delegates(_untar_data)
def untar_drive(fid, name, dest, **kwargs):
    '''download https://drive.google.com/uc?id={fid} to `dest/name.tar.gz` and 
    extract it to `dest/name`. Will not download if folder exist.'''
    return _untar_data(fid, name, dest, _drive_dl, **kwargs)

In [ ]:
#export
def _general_dl(url, fname):
    tvdu.download_url(url=url, root=fname.parent, filename=fname.name)

In [ ]:
#export
def _wget_dl(url, fname):
    command = ['wget',str(url),'-O',str(fname.absolute())]
    print(
f'''Running wget, this will not show any output and might take some time. 
If you wish, you can run it youself instead using the following command:
>>> {" ".join(command)}''')
    subprocess.run(command, check=True)

In [ ]:
#export 
@delegates(_untar_data)
def untar_general(url, name, dest, **kwargs):
    '''download `url` to `dest/name.tar.gz` using torchvision.datasets.utils.download_url
    and extract it to `dest/name`. Will not download if folder exist.'''
    return _untar_data(url, name, dest, _general_dl, **kwargs)

@delegates(_untar_data)
def untar_wget(url, name, dest, **kwargs):
    '''download `url` to `dest/name.tar.gz` using wget and extract it to `dest/name`. Will not download if folder exist.'''
    return _untar_data(url, name, dest, _wget_dl, **kwargs)

@wraps(untar_wget)
def untar_dropbox(url, name, dest, **kwargs):
    assert url.endswith('dl=0') or url.endswith('dl=1'), 'dropbox links need to end with dl=0 (single file) or dl=1(folder)'
    return untar_wget(url, name, dest, **kwargs)

In [ ]:
from nbdev.sync import notebook2script
notebook2script()

Converted 00_functional.ipynb.
Converted 01_images.ipynb.
Converted 02_fastdl.ipynb.
Converted 03_datadownload.ipynb.
Converted 04_random.ipynb.
Converted 05_tensor.ipynb.
Converted index.ipynb.
